Copy the file **spx_dte_0.csv.gz** located in https://github.com/crapher/medium/tree/main/23.OptionsStrategy/data to your colab folder

In [1]:
import warnings
import numpy as np
import pandas as pd

from tqdm import tqdm

In [2]:
# Constants
DEBUG = 1
OPTIONS_FILE='./spx_dte_0.csv.gz'

# TastyTrade
OPEN_FEES_PER_CONTRACT = 1.00

# Schwab
#OPEN_FEES_PER_CONTRACT = 0.65

CONTRACTS = 10

HOUR_OPEN = 10
MINUTE_OPEN = 30
STRIKES = 15
DIST_BETWEEN_STRIKES = 5

MAX_CHANGE_BEARISH = -0.10
MIN_CHANGE_BULLISH = 0.10
MIN_DAY_CHANGE_BULLISH = -0.15
MAX_DAY_CHANGE_BEARISH = 0.15
MIN_STRIKE_PERCENTAGE = 0.15

In [3]:
### Configuration ###
warnings.filterwarnings("ignore")
np.set_printoptions(suppress=True)
pd.options.mode.chained_assignment = None

In [4]:
### Helper functions ###
def get_vertical_option_kind(df, max_change_bearish, min_change_bullish, max_day_change_bearish, min_day_change_bullish):

    changes = 100 * (df['close_underlying'].values / df['close_underlying'].values[0] - 1)
    change = changes[-1]

    min_change = min(changes)
    max_change = max(changes)

    if change < max_change_bearish and max_change < max_day_change_bearish:
        return 'C' # Sell Call Vertical
    elif change > min_change_bullish and min_change > min_day_change_bullish:
        return 'P' # Sell Put Vertical
    else:
        return ''

def get_vertical_strikes(df, kind, strikes, dist_between_strikes):

    strike_up = int(df['close_underlying'].values[0] + (dist_between_strikes - df['close_underlying'].values[0]) % dist_between_strikes)
    strike_down = int(df['close_underlying'].values[0] - (df['close_underlying'].values[0] % dist_between_strikes))

    if kind == 'P':
        sell_strike = strike_down - strikes
        buy_strike  = sell_strike - dist_between_strikes
    else:
        sell_strike = strike_up + strikes
        buy_strike  = sell_strike + dist_between_strikes

    return (buy_strike, sell_strike)

def get_vertical_result(df, kind, buy_strike, sell_strike, min_strike_percentage):

    # Filter strikes
    df = df[(df['kind'] == kind) & ((df['strike'] == buy_strike) | (df['strike'] == sell_strike))]

    # Open Vertical
    df = df.sort_values('date', ascending=True)

    stock_price = df['close_underlying'].values[0]
    buy_price   = df[(df['strike'] == buy_strike)  & (df['kind'] == kind)]['close'].values[0]
    sell_price  = df[(df['strike'] == sell_strike) & (df['kind'] == kind)]['close'].values[0]

    if (-buy_price + sell_price) / abs(buy_strike - sell_strike) < min_strike_percentage:
        return None

    open_cost  = -100 * buy_price
    open_cost +=  100 * sell_price
    open_cost *= CONTRACTS

    if DEBUG:
        tqdm.write(f' OPEN {kind} ({stock_price:6.2f}): SELL {sell_strike} @ {sell_price:6.2f} -  BUY {buy_strike} @ {buy_price:6.2f} - TOTAL: {(-buy_price + sell_price):6.2f}')

    # Close Vertical (Prices are 0 if they are OTM/ATM or the difference between the strike and the underlying's close price if they are ITM)
    stock_price = df['close_underlying'].values[-1]
    sell_price = 0 if (kind == 'P' and buy_strike <= stock_price) or (kind == 'C' and buy_strike >= stock_price) else abs(buy_strike - stock_price)
    buy_price = 0 if (kind == 'P' and sell_strike <= stock_price) or (kind == 'C' and sell_strike >= stock_price) else abs(sell_strike - stock_price)

    if DEBUG:
        tqdm.write(f'CLOSE {kind} ({stock_price:6.2f}):  BUY {sell_strike} @ {buy_price:6.2f} - SELL {buy_strike} @ {sell_price:6.2f} - TOTAL: {(-buy_price + sell_price):6.2f}')

    close_cost = -100 * buy_price
    close_cost += 100 * sell_price
    close_cost *= CONTRACTS

    if DEBUG:
        tqdm.write(f'**************** Processed Date: {df["date"].dt.date.values[0]} - Result: {open_cost + close_cost:8.2f} ****************')

    return open_cost + close_cost

In [5]:
### Read File ###
df = pd.read_csv(OPTIONS_FILE, header=0)
df['date'] = pd.to_datetime(df['date'])

In [6]:
### Get all the expiration dates ###
expirations = df['expire_date'].unique().tolist()

In [7]:
### Get P&L by day ###
pnl = []

df_trend = df[(df.date.dt.hour < HOUR_OPEN) | ((df.date.dt.hour == HOUR_OPEN) & (df.date.dt.minute < MINUTE_OPEN))]
max_date_trend = df_trend.date.max()
df_open = df_trend[(df_trend.date.dt.hour == max_date_trend.hour) & (df_trend.date.dt.minute == max_date_trend.minute)]
df_cost = df[(df.date.dt.hour > max_date_trend.hour) | ((df.date.dt.hour == max_date_trend.hour) & (df.date.dt.minute >= max_date_trend.minute))]

df_trend = df_trend.groupby(by='expire_date')
df_cost = df_cost.groupby(by='expire_date')
df_open = df_open.groupby(by='expire_date')

for expiration in tqdm(expirations):

    try:
        kind = get_vertical_option_kind(df_trend.get_group(expiration), MAX_CHANGE_BEARISH, MIN_CHANGE_BULLISH, MAX_DAY_CHANGE_BEARISH, MIN_DAY_CHANGE_BULLISH)

        if kind == '':
            if DEBUG:
                tqdm.write(f' Processed Date: {expiration} - Result: -- NA -- '.center(84, '*'))

            continue

        (buy_strike, sell_strike) = get_vertical_strikes(df_open.get_group(expiration), kind, STRIKES, DIST_BETWEEN_STRIKES)
        gross_pnl = get_vertical_result(df_cost.get_group(expiration), kind, buy_strike, sell_strike, MIN_STRIKE_PERCENTAGE)

        if gross_pnl is None:

            if DEBUG:
                tqdm.write(f' Processed Date: {expiration} - Result: -- UV -- '.center(84, '*'))

            continue

        net_pnl = gross_pnl - CONTRACTS * OPEN_FEES_PER_CONTRACT * 2
        pnl.append([expiration, gross_pnl, net_pnl])

    except Exception as ex:
        if DEBUG:
            tqdm.write(f' Processed Date: {expiration} - Result: -- ER -- '.center(84, '*'))

  2%|▏         | 6/309 [00:00<00:11, 27.24it/s]

 OPEN C (4596.88): SELL 4615 @   7.60 -  BUY 4620 @   6.00 - TOTAL:   1.60
CLOSE C (4669.02):  BUY 4615 @  54.02 - SELL 4620 @  49.02 - TOTAL:  -5.00
**************** Processed Date: 2022-01-10 - Result: -3400.00 ****************
 OPEN P (4739.95): SELL 4720 @   4.90 -  BUY 4715 @   4.00 - TOTAL:   0.90
CLOSE P (4726.55):  BUY 4720 @   0.00 - SELL 4715 @   0.00 - TOTAL:   0.00
**************** Processed Date: 2022-01-12 - Result:   900.00 ****************
 OPEN P (4646.20): SELL 4630 @   9.40 -  BUY 4625 @   7.94 - TOTAL:   1.46
CLOSE P (4664.09):  BUY 4630 @   0.00 - SELL 4625 @   0.00 - TOTAL:   0.00
**************** Processed Date: 2022-01-14 - Result:  1460.00 ****************
 OPEN C (4319.28): SELL 4335 @  16.90 -  BUY 4340 @  14.92 - TOTAL:   1.98
CLOSE C (4428.63):  BUY 4335 @  93.63 - SELL 4340 @  88.63 - TOTAL:  -5.00
**************** Processed Date: 2022-01-28 - Result: -3020.00 ****************
 OPEN P (4571.47): SELL 4555 @   7.49 -  BUY 4550 @   5.90 - TOTAL:   1.59
CLOSE

  3%|▎         | 9/309 [00:00<00:11, 26.26it/s]

**************** Processed Date: 2022-02-16 - Result: -3200.00 ****************
 OPEN C (4291.92): SELL 4310 @  12.35 -  BUY 4315 @  11.42 - TOTAL:   0.93
CLOSE C (4227.96):  BUY 4310 @   0.00 - SELL 4315 @   0.00 - TOTAL:   0.00
**************** Processed Date: 2022-02-23 - Result:   930.00 ****************
 OPEN C (4303.45): SELL 4320 @  14.40 -  BUY 4325 @  12.10 - TOTAL:   2.30
CLOSE C (4329.36):  BUY 4320 @   9.36 - SELL 4325 @   4.36 - TOTAL:  -5.00
**************** Processed Date: 2022-03-04 - Result: -2700.00 ****************
 OPEN C (4266.02): SELL 4285 @  14.00 -  BUY 4290 @  11.94 - TOTAL:   2.06
CLOSE C (4208.31):  BUY 4285 @   0.00 - SELL 4290 @   0.00 - TOTAL:   0.00
**************** Processed Date: 2022-03-07 - Result:  2060.00 ****************
 OPEN C (4262.05): SELL 4280 @   9.90 -  BUY 4285 @   8.57 - TOTAL:   1.33
CLOSE C (4203.91):  BUY 4280 @   0.00 - SELL 4285 @   0.00 - TOTAL:   0.00
**************** Processed Date: 2022-03-11 - Result:  1330.00 ****************


  5%|▍         | 15/309 [00:00<00:11, 26.50it/s]

...

CLOSE C (5021.42):  BUY 5075 @   0.00 - SELL 5080 @   0.00 - TOTAL:   0.00
**************** Processed Date: 2024-04-17 - Result:  1700.00 ****************
 OPEN C (4979.17): SELL 4995 @   6.32 -  BUY 5000 @   4.90 - TOTAL:   1.42
CLOSE C (5011.48):  BUY 4995 @  16.48 - SELL 5000 @  11.48 - TOTAL:  -5.00
**************** Processed Date: 2024-04-22 - Result: -3580.00 ****************
 OPEN P (5059.61): SELL 5040 @   5.27 -  BUY 5035 @   4.30 - TOTAL:   0.97
CLOSE P (5069.41):  BUY 5040 @   0.00 - SELL 5035 @   0.00 - TOTAL:   0.00
**************** Processed Date: 2024-04-23 - Result:   970.00 ****************
 OPEN P (5096.85): SELL 5080 @   6.60 -  BUY 5075 @   5.50 - TOTAL:   1.10
CLOSE P (5098.69):  BUY 5080 @   0.00 - SELL 5075 @   0.00 - TOTAL:   0.00
**************** Processed Date: 2024-04-26 - Result:  1100.00 ****************


 97%|█████████▋| 300/309 [00:22<00:00, 20.12it/s]

 OPEN C (5022.94): SELL 5040 @  12.63 -  BUY 5045 @  11.00 - TOTAL:   1.63
CLOSE C (5017.57):  BUY 5040 @   0.00 - SELL 5045 @   0.00 - TOTAL:   0.00
**************** Processed Date: 2024-05-01 - Result:  1630.00 ****************
 OPEN C (5034.40): SELL 5050 @   8.00 -  BUY 5055 @   6.17 - TOTAL:   1.83
CLOSE C (5065.36):  BUY 5050 @  15.36 - SELL 5055 @  10.36 - TOTAL:  -5.00
**************** Processed Date: 2024-05-02 - Result: -3170.00 ****************
 OPEN P (5158.50): SELL 5140 @   3.24 -  BUY 5135 @   2.47 - TOTAL:   0.77
CLOSE P (5180.70):  BUY 5140 @   0.00 - SELL 5135 @   0.00 - TOTAL:   0.00
**************** Processed Date: 2024-05-06 - Result:   770.00 ****************
****************** Processed Date: 2024-05-08 - Result: -- UV -- *******************
****************** Processed Date: 2024-05-13 - Result: -- UV -- *******************
****************** Processed Date: 2024-05-14 - Result: -- UV -- *******************


100%|██████████| 309/309 [00:22<00:00, 13.71it/s]

****************** Processed Date: 2024-05-20 - Result: -- UV -- *******************
****************** Processed Date: 2024-05-21 - Result: -- UV -- *******************
 OPEN C (5313.66): SELL 5330 @   4.22 -  BUY 5335 @   2.99 - TOTAL:   1.23
CLOSE C (5268.33):  BUY 5330 @   0.00 - SELL 5335 @   0.00 - TOTAL:   0.00
**************** Processed Date: 2024-05-23 - Result:  1230.00 ****************
 OPEN P (5295.12): SELL 5280 @   3.40 -  BUY 5275 @   2.55 - TOTAL:   0.85
CLOSE P (5304.70):  BUY 5280 @   0.00 - SELL 5275 @   0.00 - TOTAL:   0.00
**************** Processed Date: 2024-05-24 - Result:   850.00 ****************
****************** Processed Date: 2024-05-28 - Result: -- UV -- *******************
 OPEN P (5276.50): SELL 5260 @   3.20 -  BUY 5255 @   2.26 - TOTAL:   0.94
CLOSE P (5265.78):  BUY 5260 @   0.00 - SELL 5255 @   0.00 - TOTAL:   0.00
**************** Processed Date: 2024-05-29 - Result:   940.00 ****************


In [8]:
### Generate result ###

# Calculate wins and losses
df_result = pd.DataFrame(pnl, columns=['expire_date','gross_result','net_result'])
df_result['wins'] = np.where(df_result['net_result'] > 0, 1, 0)
df_result['losses'] = np.where(df_result['net_result'] < 0, 1, 0)

In [9]:
# Show the parameters
print(f' PARAMETERS '.center(70, '*'))
print(f'* Hour Open: {HOUR_OPEN}')
print(f'* Minute Open: {MINUTE_OPEN}')
print(f'* Strikes From Price: {STRIKES}')
print(f'* Distance Between Strikes: {DIST_BETWEEN_STRIKES}')
print(f'* Max % Change To Open A Bearish Trade: {MAX_CHANGE_BEARISH}')
print(f'* Min % Change To Open A Bullish Trade: {MIN_CHANGE_BULLISH}')
print(f'* Max Day Change To Open A Bearish Trade: {MAX_DAY_CHANGE_BEARISH}')
print(f'* Min Day Change To Open A Bullish Trade: {MIN_DAY_CHANGE_BULLISH}')
print(f'* Min Strike Percentage: {MIN_STRIKE_PERCENTAGE}')

# Show the Total Result
print(f' SUMMARIZED RESULT '.center(70, '*'))
print(f'* Trading Days: {len(df_result)}')
print(f'* Gross PnL: $ {df_result["gross_result"].sum():.2f}')
print(f'* Net PnL: $ {df_result["net_result"].sum():.2f}')
print(f'* Win Rate: {100 * (df_result["wins"].sum() / (df_result["wins"].sum() + df_result["losses"].sum())):.2f} %')

# Show The Monthly Result
print(f' MONTHLY DETAIL RESULT '.center(70, '*'))
df_monthly = df_result[['expire_date','gross_result','net_result','wins','losses']]
df_monthly['year_month'] = df_monthly['expire_date'].str[0:7]
df_monthly = df_monthly.drop('expire_date', axis=1)
df_monthly = df_monthly.groupby(['year_month']).sum()

print(df_monthly)

# Show The Yearly Result
print(f' YEARLY DETAIL RESULT '.center(70, '*'))
df_yearly = df_result[['expire_date','gross_result','net_result','wins','losses']]
df_yearly['year'] = df_yearly['expire_date'].str[0:4]
df_yearly = df_yearly.drop('expire_date', axis=1)
df_yearly = df_yearly.groupby(['year']).sum()

print(df_yearly)


***************************** PARAMETERS *****************************
* Hour Open: 10
* Minute Open: 30
* Strikes From Price: 15
* Distance Between Strikes: 5
* Max % Change To Open A Bearish Trade: -0.1
* Min % Change To Open A Bullish Trade: 0.1
* Max Day Change To Open A Bearish Trade: 0.15
* Min Day Change To Open A Bullish Trade: -0.15
* Min Strike Percentage: 0.15
************************* SUMMARIZED RESULT **************************
* Trading Days: 262
* Gross PnL: $ 111680.00
* Net PnL: $ 106440.00
* Win Rate: 82.82 %
*********************** MONTHLY DETAIL RESULT ************************
            gross_result  net_result  wins  losses
year_month                                        
2022-01          -4060.0     -4140.0     2       2
2022-02            220.0       140.0     3       1
2022-03           7290.0      7130.0     7       1
2022-04          12870.0     12670.0    10       0
2022-05           1110.0       910.0     7       3
2022-06          14540.0     14340.0   